In [9]:
#final predictor
import os 
import librosa
import pandas as pd
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tqdm import tqdm


In [10]:
GENRE_LIST=['metal','hiphop','reggae','rock','classical','jazz','blues','country','pop','disco']

In [11]:
#Todo modify method to return dataframe
def CreateDataSet(directory):
    label=[]
    spectralcentroids=[]
    zerocrossing=[]
    spectralbandwidthdata=[]
    spectralrolloffs=[]
    mfc=[]
    
    for folder in directory:
        
        for filename in tqdm(os.listdir(folder)):
            music_name=os.path.join(folder,filename)
            try:
                X, sample_rate = librosa.load(music_name, res_type='kaiser_fast')
                
                label.append(folder)
                mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
                mfc.append(mfccs)
                
                ##zero crossing
                zerocrossings = librosa.feature.spectral_centroid(y=X, sr=sample_rate , hop_length=512)
                centroid = np.mean(np.array(zerocrossings))
                std = np.std(np.array(zerocrossings))
                zerocrossing.append((centroid,std))
                
                ##spectral bandwidth
                spectralbandwidth = librosa.feature.spectral_bandwidth(y=X, sr=sample_rate , hop_length=512)
                centroid = np.mean(np.array(spectralbandwidth))
                std = np.std(np.array(spectralbandwidth))
                spectralbandwidthdata.append((centroid,std))
                
                ##spectral centroid 
                spectralcentroid = librosa.feature.spectral_centroid(y=X, sr=sample_rate , hop_length=512)
                centroid = np.mean(np.array(spectralcentroid))
                std = np.std(np.array(spectralcentroid))
                spectralcentroids.append((centroid,std))
                
                #3 spectral rolloff
                spectralrolloff = librosa.feature.spectral_rolloff(y=X, sr=sample_rate , hop_length=512)
                centroid = np.mean(np.array(spectralrolloff))
                std = np.std(np.array(spectralrolloff))
                spectralrolloffs.append((centroid,std))
                
            except Exception as e:
                print("Error encountered while parsing file: ", e)
                return None
    return np.array(label), np.array(mfc), np.array(zerocrossing), np.array(spectralbandwidthdata), np.array(spectralcentroids), np.array(spectralrolloffs)

In [58]:
label, mfc, zero, spectral, centroid, rolloff = CreateDataSet(GENRE_LIST)

100%|██████████| 100/100 [00:31<00:00,  3.15it/s]


In [59]:
centroid.shape

(1000, 2)

In [13]:
lb = LabelEncoder()
y = np_utils.to_categorical(lb.fit_transform(label))

In [27]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Conv2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

def createModel(inputshape):
    if inputshape is 40:
        num_labels = y.shape[1]
        # build model
        model = Sequential()
        model.add(Dense(256, input_shape=(40,)))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(256))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(num_labels))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    else:
        model = Sequential()
        model.add(Dense(32, input_shape=(2,)))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(10))
        model.add(Activation('softmax'))
        model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [28]:
mfcc = createModel(40)
mfcc.load_weights("mfcc.h5")

zerocrossing = createModel(2)
zerocrossing.load_weights("zero_crossing")

spectralbandwith = createModel(2)
spectralbandwith.load_weights("Spectral_bandwidth")

spectralcentroid = createModel(2)
spectralcentroid.load_weights("Spectral_Centroid")

spectralrolloff = createModel(2)
spectralrolloff.load_weights("Spectral_rolloff")

In [29]:
def getPath(directory):
    label=[]
    musicfile=[]
    for folder in directory:
        for filename in tqdm(os.listdir(folder)):
            music_name=os.path.join(folder,filename)
            musicfile.append(music_name)
            label.append(folder)
    return np.array(music_name), np.array(label)

In [30]:
path, label = getPath(GENRE_LIST)

100%|██████████| 100/100 [00:00<00:00, 227580.25it/s]


In [79]:
output1 = zerocrossing.predict(zero)
output2 = mfcc.predict(mfc)
output3 = spectralbandwith.predict(spectral)
output4 = spectralcentroid.predict(centroid)
output5 = spectralrolloff.predict(rolloff)

In [106]:
def predictionarray(data1, data2, data3, data4, data5):
    pred=[]
    for row1, row2, row3, row4, row5 in zip(data1,data2,data3,data4,data5):
        pred.append([np.argmax(row1),np.argmax(row2),np.argmax(row3),np.argmax(row4),np.argmax(row5)])
    return np.array(pred)

In [111]:
finaloutput = predictionarray(output1, output2, output3, output4, output5)

In [112]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(finaloutput, label, test_size=0.33, random_state=42)

In [114]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( X_train, y_train )

In [116]:
forest.predict(X_test)

array(['classical', 'classical', 'classical', 'classical', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'hiphop', 'metal', 'hiphop', 'classical', 'metal', 'classical',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'hiphop', 'classical', 'classical', 'classical', 'classical',
       'classical', 'hiphop', 'hiphop', 'country', 'classical', 'hiphop',
       'classical', 'classical', 'classical', 'classical', 'hiphop',
       'classical', 'classical', 'metal', 'classical', 'hiphop',
       'classical', 'hiphop', 'classical', 'classical', 'classical',
       'classical', 'classical', 'hiphop', 'classical', 'classical',
       'classical', 'classical', 'metal', 'metal', 'classical', 'metal',
       'classical', 'classical', 'hiphop', 'classical', 'classical',
       'country', 'hiphop', 'classical', 'classical', 'hiphop', 'metal',
       'classical', 'classical', 'classical', 'classical', 'classical',
       'cla

In [117]:
print(y_test)

['jazz' 'country' 'country' 'blues' 'classical' 'blues' 'blues' 'jazz'
 'pop' 'hiphop' 'pop' 'metal' 'blues' 'disco' 'disco' 'pop' 'reggae' 'pop'
 'country' 'rock' 'jazz' 'hiphop' 'rock' 'jazz' 'reggae' 'reggae' 'hiphop'
 'disco' 'disco' 'disco' 'rock' 'hiphop' 'blues' 'classical' 'rock'
 'hiphop' 'blues' 'jazz' 'disco' 'metal' 'rock' 'metal' 'rock' 'disco'
 'reggae' 'reggae' 'disco' 'blues' 'classical' 'pop' 'jazz' 'pop' 'reggae'
 'jazz' 'metal' 'metal' 'blues' 'metal' 'reggae' 'pop' 'reggae' 'metal'
 'jazz' 'reggae' 'blues' 'blues' 'blues' 'pop' 'metal' 'jazz' 'hiphop'
 'classical' 'jazz' 'country' 'reggae' 'reggae' 'blues' 'metal' 'jazz'
 'rock' 'blues' 'metal' 'reggae' 'metal' 'rock' 'jazz' 'disco' 'country'
 'country' 'rock' 'rock' 'classical' 'jazz' 'reggae' 'country' 'classical'
 'country' 'metal' 'metal' 'pop' 'country' 'rock' 'disco' 'reggae' 'blues'
 'blues' 'jazz' 'disco' 'hiphop' 'disco' 'jazz' 'pop' 'disco' 'pop'
 'metal' 'classical' 'hiphop' 'disco' 'jazz' 'reggae' 'pop' 